# Credmark - Uniswap V3 Liquidity Explorer

- 2022.7.29

In [ ]:
from decimal import Decimal
import math

from src.sqrtmath import SqrtPriceMath
from src.v3_liquidity import *

smath = SqrtPriceMath()

In [ ]:
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'},
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf cmf_param

### Load from Ledger or file


In [ ]:
load_from_ledger = False

if load_from_ledger:
    pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

    with pool.ledger.events.Flash as q:
        df_flash = q.select(columns=q.columns,
                        order_by=q.EVT_BLOCK_NUMBER,
                        limit=5000).to_dataframe()

    with pool.ledger.events.Mint as q:
        df_mint = q.select(columns=q.columns,
                        order_by=q.EVT_BLOCK_NUMBER.comma_(q.EVT_INDEX),
                        where=q.EVT_BLOCK_NUMBER.le(12384601)
                        ).to_dataframe()

    with pool.ledger.events.Burn as q:
        df_burn = q.select(columns=q.columns,
                        order_by=q.EVT_BLOCK_NUMBER.comma_(q.EVT_INDEX),
                        where=q.EVT_BLOCK_NUMBER.le(12384601)).to_dataframe()

    with pool.ledger.events.Swap as q:
        df_swap = q.select(columns=q.columns,
                        order_by=q.EVT_BLOCK_NUMBER.comma_(q.EVT_INDEX),
                        where=q.EVT_BLOCK_NUMBER.le(12384601)).to_dataframe()

    df_flash.to_csv('csv/uni_flash.csv', index=False)
    df_mint.to_csv('csv/uni_mint.csv', index=False)
    df_burn.to_csv('csv/uni_burn.csv', index=False)
    df_swap.to_csv('csv/uni_swap.csv', index=False)

    with pool.ledger.events.Mint as q:
        df_mint_min_max = q.select(aggregates=[(q.TICKLOWER.min_(), 'lower_min'),
                                               (q.TICKUPPER.max_(), 'upper_max'),
                                               (q.EVT_BLOCK_NUMBER.min_(), 'block_number_min'),
                                               (q.EVT_BLOCK_NUMBER.max_(), 'block_number_max')]
                                ).to_dataframe()

    with pool.ledger.events.Burn as q:
        df_burn_min_max = q.select(aggregates=[(q.TICKLOWER.min_(), 'lower_min'),
                                               (q.TICKUPPER.max_(), 'upper_max'),
                                               (q.EVT_BLOCK_NUMBER.min_(), 'block_number_min'),
                                               (q.EVT_BLOCK_NUMBER.max_(), 'block_number_max')]
                                ).to_dataframe()   

    df_mint_min_max.to_csv('csv/df_mint_min_max.csv', index=False)
    df_burn_min_max.to_csv('csv/df_burn_min_max.csv', index=False)
else:
    df_flash = pd.read_csv('csv/uni_flash.csv')
    df_mint = pd.read_csv('csv/uni_mint.csv')
    df_burn = pd.read_csv('csv/uni_burn.csv')
    df_swap = pd.read_csv('csv/uni_swap.csv')

    df_mint_min_max = pd.read_csv('csv/df_mint_min_max.csv')
    df_burn_min_max = pd.read_csv('csv/df_burn_min_max.csv')    

## Transform

1. Mint/Burn may not change liquidity, such as when amount = 0 (there could be more)
2. Swap may change the liquidity

In [ ]:
@Model.describe(
    slug='jit.liquidity',
    version='1.0',
    output=dict)
class JitLiquidity(Model):
    def run(self, input):
        pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
        current_liquidity = pool.functions.liquidity().call()
        slot0 = pool.functions.slot0().call()
        current_tick = slot0[1]
        token0_addr = pool.functions.token0().call()
        token1_addr = pool.functions.token1().call()
        token0 = Token(address=Address(token0_addr).checksum)
        token1 = Token(address=Address(token1_addr).checksum)
        token0_bal = token0.balance_of(pool.address.checksum)
        token1_bal = token1.balance_of(pool.address.checksum)
        return {'liquidity': current_liquidity, 'tick': current_tick, 'token0_bal': token0_bal, 'token1_bal': token1_bal}

context.remove_model_by_slug('jit.liquidity')
context.add_model(JitLiquidity)

# Check whether the model has been loaded
assert 'jit.liquidity' in model_loader.loaded_model_version_lists()

In [ ]:
df_mint_sel = (df_mint
    [['evt_block_number', 'evt_index', 'evt_tx_hash', 'inp_owner', 'inp_tickLower', 'inp_tickUpper', 'inp_amount', 'inp_amount0', 'inp_amount1']]
    .assign(inp_amount1 = lambda x: x.inp_amount1.apply(int))
)
df_mint_sel

In [ ]:
df_burn_sel = (df_burn
    [['evt_block_number', 'evt_index', 'evt_tx_hash', 'inp_owner', 'inp_tickLower', 'inp_tickUpper', 'inp_amount', 'inp_amount0', 'inp_amount1']]
    .assign(inp_amount=lambda x: -x.inp_amount,
            inp_amount0=lambda x: -x.inp_amount0,
            inp_amount1=lambda x: -x.inp_amount1)
)
df_burn_sel

In [ ]:
df_swap.dtypes

In [ ]:
(df_swap
    [['evt_block_number', 'evt_index', 'evt_tx_hash', 'inp_recipient', 'inp_liquidity', 'inp_amount0', 'inp_amount1', 'inp_tick', 'inp_sqrtPriceX96']]
    .assign(inp_amount1 = lambda x: x.inp_amount1.apply(int),
            inp_sqrtPriceX96_d = lambda x: x.inp_sqrtPriceX96.apply(int))
    .assign(price = lambda x: x.inp_sqrtPriceX96_d * x.inp_sqrtPriceX96_d / (2 ** 192),
            price_tick = lambda x: (1.0001 ** x.inp_tick).apply(int),
            price_sqrt = lambda x: (1.0001 ** (x.inp_tick / 2) * (2 ** 96)).apply(int))
)

In [77]:
df_swap_sel = (
    df_swap
        [['evt_block_number', 'evt_index', 'evt_tx_hash', 'inp_recipient', 'inp_liquidity', 'inp_amount0', 'inp_amount1', 'inp_tick']]
        .assign(inp_amount1 = lambda x: x.inp_amount1.apply(int))
        .sort_values(['evt_block_number', 'evt_index'])
        .reset_index(drop=True)
)

# add inp_amount as the impact to laddressiquidity from Swaps (summed up per block)
df_swap_sel = df_swap_sel.assign(inp_amount=int(0))

evt_block_number = None
for n,r in df_swap_sel.iterrows():
    tick_info = context.models(r.evt_block_number-1).jit.liquidity()
    liquidity_diff_block = r.inp_liquidity - tick_info['liquidity']

    if evt_block_number is None or evt_block_number != r.evt_block_number:
        evt_block_number = r.evt_block_number 
        df_swap_sel.loc[n, 'inp_amount'] = liquidity_diff_block
    else:
        if liquidity_diff_block == df_swap_sel.loc[n-1, 'inp_amount']:
            df_swap_sel.loc[n, 'inp_amount'] = 0
        else:
            df_swap_sel.loc[n, 'inp_amount'] = r.inp_liquidity - tick_info['liquidity']
df_swap_sel

KeyboardInterrupt: 

In [ ]:
# swaps that increases the liquidity
df_swap_sel.loc[df_swap_sel.inp_amount > 0, :]

In [ ]:
# swaps that descreases the liquidity
df_swap_sel.loc[df_swap_sel.inp_amount < 0, :]

In [ ]:
df_mint_burn_swap = (
    pd.concat([df_mint_sel.assign(evt_type='mint', inp_liquidity=pd.NA, inp_tick=pd.NA),
               df_burn_sel.assign(evt_type='burn', inp_liquidity=pd.NA, inp_tick=pd.NA),
               df_swap_sel.assign(evt_type='swap', inp_tickLower=pd.NA, inp_tickUpper=pd.NA)])
    .sort_values(['evt_block_number', 'evt_index'])
    .reset_index(drop=True)
    .assign(liquidity = lambda x: x.inp_amount.cumsum())
    )
df_mint_burn_swap

In [ ]:
tick_infos = [
    context.models(r.evt_block_number).jit.liquidity().values()
    for _,r in df_mint_burn_swap.iterrows()
]

liquidity_data = dict(zip(['pool_liquidity', 'current_tick', 'token0_bal', 'token1_bal'], zip(*tick_infos)))
df_mint_burn_swap = df_mint_burn_swap.assign(**liquidity_data)

df_mint_burn_swap.to_csv('csv/df_mint_burn_swap.csv', index=False)
df_mint_burn_swap

### 1. First Mint - add liquidity

In [ ]:
(df_mint
    [['evt_block_number', 'evt_index', 'evt_tx_hash', 'inp_owner', 'inp_tickLower', 'inp_tickUpper', 'inp_amount', 'inp_amount0', 'inp_amount1']]
    .loc[0]
)

In [ ]:
param = {'chain_id': 1,
 'block_number': 12376729,
 'model_loader_path': [],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': '*',
 'register_utility_global': True}

context2, model_loader2 = %cmf param

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

current_liquidity = pool.functions.liquidity().call()
tick_spacing = pool.functions.tickSpacing().call()

slot0 = pool.functions.slot0().call()
current_sqrtPriceX96 = slot0[0]
current_tick = slot0[1]
current_price = tick_to_price(current_tick)

token0_addr = pool.functions.token0().call()
token1_addr = pool.functions.token1().call()
token0 = Token(address=Address(token0_addr).checksum)
token1 = Token(address=Address(token1_addr).checksum)
token0_bal = token0.balance_of(pool.address.checksum)
token1_bal = token1.balance_of(pool.address.checksum)

print((token0_bal, token1_bal))

scale_multiplier = (10 ** (token0.decimals - token1.decimals))
adjusted_current_price = current_price * scale_multiplier
current_sqrt_price = current_sqrtPriceX96 * current_sqrtPriceX96 / (2 ** 192) * scale_multiplier

(current_tick,
 math.log(current_sqrtPriceX96 / (2 ** 96)) / math.log(1.0001) * 2,
 smath.getSqrtRatioAtTick(current_tick),
 current_sqrtPriceX96,
 Decimal((1.0001 ** (current_tick/2) * (2 ** 96))),
 current_price * scale_multiplier,
 adjusted_current_price,
 1 / adjusted_current_price,
 current_sqrt_price,
  token0.symbol,
 token1.symbol
)
 

#### In-tick liquidity

In [ ]:
display(f'{slot0=}, {tick_spacing=}, {current_tick=}, {current_liquidity=}')

# Compute the tick range near the current tick
current_tick_bottom = current_tick // tick_spacing * tick_spacing
current_tick_top = current_tick_bottom + tick_spacing
display((current_tick_bottom, current_tick, current_tick_top))
assert current_tick_bottom <= current_tick <= current_tick_top

print('Price from tick:', current_price, 'Price from sqrt:', current_sqrtPriceX96 * current_sqrtPriceX96 / (2 ** 192))

sa = tick_to_price(current_tick_bottom // 2)
sb = tick_to_price(current_tick_top // 2)
sp = current_price ** 0.5

in_tick_amount0 = current_liquidity * (1 / sp - 1 / sb)
in_tick_amount1 = current_liquidity * (sp - sa)

display((f'{current_liquidity=}', f'{in_tick_amount0=}', f'{in_tick_amount1=}'))
display(('Test equality:', (in_tick_amount0 + current_liquidity / sb) * (in_tick_amount1 + current_liquidity * sa) - float(current_liquidity * current_liquidity)))
display(('token0_in_tick', in_tick_amount0 / token0_bal, 'token1_in_tick', in_tick_amount1 / token1_bal))


#### In-range liquidity/amount

In [ ]:
liquidity = pool.functions.liquidity().call()

tick_bottom = 191150
tick_top = 198080

tick = current_tick

sa = tick_to_price(tick_bottom // 2)
sb = tick_to_price(tick_top // 2)
sp = current_price ** 0.5

in_tick_amount0 = int(liquidity * (1 / sp - 1 / sb))
in_tick_amount1 = int(liquidity * (sp - sa))

display((f'{sa=}, {sb=}, {sp=}, {liquidity=}', f'{in_tick_amount0=}', f'{in_tick_amount1=}'))
test_1 = (in_tick_amount0 + liquidity / sb) * (in_tick_amount1 + liquidity * sa)
test_2 = (liquidity * liquidity)
display(('Test equality:', tick, test_1 / test_2 - 1, test_1 - test_2) )
display(('token0_in_tick', in_tick_amount0 / token0_bal, 'token1_in_tick', in_tick_amount1 / token1_bal))


### In-range liquidity/amount vs. price

In [ ]:
liquidity = pool.functions.liquidity().call()

tick_bottom = 191150
tick_top = 198080

sa = tick_to_price(tick_bottom // 2)
sb = tick_to_price(tick_top // 2)

display(sa, sb)

ax = plt.gca()    # Get current axis
ax2 = ax.twinx()  # make twin axis based on x

ticks = []
amount0 = []
amount1 = []
prices = []

for tick in [tick_bottom, (tick_bottom + current_tick) / 2,  current_tick, (current_tick + tick_top) / 2, tick_top]:
    sp = tick_to_price(tick // 2)
    
    in_tick_amount0 = int(liquidity * (1 / sp - 1 / sb))
    in_tick_amount1 = int(liquidity * (sp - sa))

    ticks.append(tick)
    amount0.append(token0.scaled(in_tick_amount0))
    amount1.append(token1.scaled(in_tick_amount1))
    prices.append(1 / (sp * sp * scale_multiplier))

    display((f'{sa=}, {sb=}, {sp=}, {liquidity=}', f'{in_tick_amount0=}', f'{in_tick_amount1=}'))
    test_1 = (in_tick_amount0 + liquidity / sb) * (in_tick_amount1 + liquidity * sa)
    test_2 = (liquidity * liquidity)
    display(('Test equality:', tick, test_1 / test_2 - 1, test_1 - test_2) )

ax.scatter(ticks, amount0)
ax.plot(ticks, amount0)
ax.text(ticks[0], amount0[0], 'token0')
ax2.scatter(ticks, amount1)
ax2.plot(ticks, amount1)
ax2.text(ticks[0], amount1[0], 'token1')

plt.show()

pd.DataFrame({'ticks': ticks, 'amount0': amount0, 'amount1': amount1, 'prices': prices})


### Full tick spacing liquidity

In [ ]:
pool_min_tick = 191150
pool_max_tick = 198080

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
 
tick_mapping = get_liquidity_in_ticks(pool, pool_min_tick, pool_max_tick)

display(tick_mapping)

token0_addr = pool.functions.token0().call()
token1_addr = pool.functions.token1().call()
token0 = Token(address=Address(token0_addr).checksum)
token1 = Token(address=Address(token1_addr).checksum)

df_pool = get_amount_in_ticks(pool, token0, token1, tick_mapping)

display((
    ('token0 balance: ', token0_bal, df_pool.token0_locked.sum(), token0_bal - df_pool.token0_locked.sum()),
    ('token1 balance: ', df_pool.token1_locked.sum(), token1_bal, df_pool.token1_locked.sum() - token1_bal),
))

df_pool.plot(x = 'tick', y = ['liquidity', 'token0_locked', 'token1_locked'], sharex=True, subplots=True, kind='line')
plt.show()

In [ ]:
display(pool.functions.slot0().call()[1])
df_pool.loc[df_pool.tick.isin([194850, 194860,194870,194880,194890]),:]

In [ ]:
plt.plot(df_pool.tick, df_pool.token0); plt.show()

### 2. After first swap

In [ ]:
param = {'chain_id': 1,
 'block_number': 12376891,
 'model_loader_path': [],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': '*',
 'register_utility_global': True}

context3, model_loader3 = %cmf param
context3.set_current()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

current_liquidity = pool.functions.liquidity().call()
tick_spacing = pool.functions.tickSpacing().call()

slot0 = pool.functions.slot0().call()
current_sqrtPriceX96 = slot0[0]
current_tick = slot0[1]
current_price = tick_to_price(current_tick)

token0_addr = pool.functions.token0().call()
token1_addr = pool.functions.token1().call()
token0 = Token(address=Address(token0_addr).checksum)
token1 = Token(address=Address(token1_addr).checksum)
token0_bal = token0.balance_of(pool.address.checksum)
token1_bal = token1.balance_of(pool.address.checksum)

print((token0_bal, token1_bal))

In [ ]:
df_mint_burn_swap_sel = df_mint_burn_swap.query('evt_block_number <= @context3.block_number')
df_mint_burn_swap_sel

In [ ]:
pool_min_tick = df_mint_burn_swap_sel.inp_tickLower.min()
pool_max_tick = df_mint_burn_swap_sel.inp_tickUpper.max()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
 
tick_mapping = get_liquidity_in_ticks(pool, pool_min_tick, pool_max_tick)

display(tick_mapping)

df_pool2 = get_amount_in_ticks(pool, token0, token1, tick_mapping)

display((
    ('token0 balance: ', token0_bal, df_pool2.token0_locked.sum(), token0_bal - df_pool2.token0_locked.sum()),
    ('token1 balance: ', df_pool2.token1_locked.sum(), token1_bal, df_pool2.token1_locked.sum() - token1_bal),
))

df_pool2.plot(x = 'tick', y = ['liquidity', 'token0_locked', 'token1_locked'], sharex=True, subplots=True, kind='line')
plt.show()

### After mint and burn

In [ ]:
param = {'chain_id': 1,
 'block_number': 12377266,
 'model_loader_path': [],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': '*',
 'register_utility_global': True}

context4, model_loader4 = %cmf param
context4.set_current()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

current_liquidity = pool.functions.liquidity().call()
tick_spacing = pool.functions.tickSpacing().call()

slot0 = pool.functions.slot0().call()
current_sqrtPriceX96 = slot0[0]
current_tick = slot0[1]
current_price = tick_to_price(current_tick)

token0_addr = pool.functions.token0().call()
token1_addr = pool.functions.token1().call()
token0 = Token(address=Address(token0_addr).checksum)
token1 = Token(address=Address(token1_addr).checksum)
token0_bal = token0.balance_of(pool.address.checksum)
token1_bal = token1.balance_of(pool.address.checksum)

print((token0_bal, token1_bal, current_liquidity))

In [ ]:
df_mint_burn_swap_sel = df_mint_burn_swap.query('evt_block_number <= @context4.block_number')
print(df_mint_burn_swap_sel.liquidity.to_list()[-1])
df_mint_burn_swap_sel

In [ ]:
pool_min_tick = df_mint_burn_swap_sel.inp_tickLower.min()
pool_max_tick = df_mint_burn_swap_sel.inp_tickUpper.max()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
 
tick_mapping = get_liquidity_in_ticks(pool, pool_min_tick, pool_max_tick)

display(tick_mapping)

df_pool2 = get_amount_in_ticks(pool, token0, token1, tick_mapping)

display((
    ('token0 balance: ', token0_bal, df_pool2.token0_locked.sum(), token0_bal - df_pool2.token0_locked.sum()),
    ('token1 balance: ', df_pool2.token1_locked.sum(), token1_bal, df_pool2.token1_locked.sum() - token1_bal),
))

df_pool2.plot(x = 'tick', y = ['liquidity', 'token0_locked', 'token1_locked'], sharex=True, subplots=True, kind='line')
plt.show()

In [ ]:
param = {'chain_id': 1,
 'block_number': 12377278,
 'model_loader_path': [],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': '*',
 'register_utility_global': True}

context5, model_loader5 = %cmf param
context5.set_current()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

current_liquidity = pool.functions.liquidity().call()
tick_spacing = pool.functions.tickSpacing().call()

slot0 = pool.functions.slot0().call()
current_sqrtPriceX96 = slot0[0]
current_tick = slot0[1]
current_price = tick_to_price(current_tick)

token0_addr = pool.functions.token0().call()
token1_addr = pool.functions.token1().call()
token0 = Token(address=Address(token0_addr).checksum)
token1 = Token(address=Address(token1_addr).checksum)
token0_bal = token0.balance_of(pool.address.checksum)
token1_bal = token1.balance_of(pool.address.checksum)

print((token0_bal, token1_bal, current_liquidity, current_tick))

In [ ]:
df_mint_burn_swap_sel = df_mint_burn_swap.query('evt_block_number <= @context5.block_number')
df_mint_burn_swap_sel

In [ ]:
pool_min_tick = df_mint_burn_swap_sel.inp_tickLower.min()
pool_max_tick = df_mint_burn_swap_sel.inp_tickUpper.max()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
 
tick_mapping = get_liquidity_in_ticks(pool, pool_min_tick, pool_max_tick)

display(tick_mapping)

df_pool2 = get_amount_in_ticks(pool, token0, token1, tick_mapping)

display((
    ('token0 balance: ', token0_bal, df_pool2.token0_locked.sum(), token0_bal - df_pool2.token0_locked.sum()),
    ('token1 balance: ', df_pool2.token1_locked.sum(), token1_bal, df_pool2.token1_locked.sum() - token1_bal),
))

df_pool2.plot(x = 'tick', y = ['liquidity', 'token0_locked', 'token1_locked'], sharex=True, subplots=True, kind='line')
plt.show()

In [ ]:
param = {'chain_id': 1,
 'block_number': 15269596,
 'model_loader_path': ['../../credmark-models-py/models'],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': '*',
 'register_utility_global': True}

context6, model_loader6 = %cmf param
context6.set_current()

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

current_liquidity = pool.functions.liquidity().call()
tick_spacing = pool.functions.tickSpacing().call()

slot0 = pool.functions.slot0().call()
current_sqrtPriceX96 = slot0[0]
current_tick = slot0[1]
current_price = tick_to_price(current_tick)

token0_addr = pool.functions.token0().call()
token1_addr = pool.functions.token1().call()
token0 = Token(address=Address(token0_addr).checksum)
token1 = Token(address=Address(token1_addr).checksum)
token0_bal = token0.balance_of(pool.address.checksum)
token1_bal = token1.balance_of(pool.address.checksum)

print((token0_bal, token1_bal, current_liquidity, current_tick))

In [ ]:
pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

pool_min_tick = min(df_burn_min_max.lower_min.min(), df_mint_min_max.lower_min.min())
pool_max_tick = max(df_burn_min_max.upper_max.max(), df_mint_min_max.upper_max.max())

liquidity_by_ticks = context.run_model('uniswap-v3.get-liquidity-by-ticks', 
                                      {"address": "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
                                      "min_tick": pool_min_tick,
                                      "max_tick": pool_max_tick})

liquidity_by_ticks


In [ ]:
pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
 
tick_mapping_1k = get_liquidity_in_ticks(pool, current_tick-1000, current_tick+1000)

df_pool_1k = get_amount_in_ticks(pool, token0, token1, tick_mapping_1k)

display((
    ('token0 balance: ', token0_bal, df_pool_1k.token0_locked.sum(), token0_bal - df_pool_1k.token0_locked.sum()),
    ('token1 balance: ', df_pool_1k.token1_locked.sum(), token1_bal, df_pool_1k.token1_locked.sum() - token1_bal),
))

df_pool_1k.plot(x = 'tick', y = ['liquidity', 'token0_locked', 'token1_locked'], sharex=True, subplots=True, kind='line')
plt.show()

assert df_pool_1k.liquidity[df_pool_1k.liquidity < 0].shape[0] == 0

In [ ]:
list(tick_mapping_1k.items())[:10]

In [ ]:
(pd
    .DataFrame([(int(k), int(v)) for k, v in out['liquidity'].items()],
               columns=['tick', 'liquidity'])
    .plot('tick', 'liquidity')
)
plt.show()


In [ ]:
pool_min_tick = min(df_burn_min_max.lower_min.min(), df_mint_min_max.lower_min.min())
pool_max_tick = max(df_burn_min_max.upper_max.max(), df_mint_min_max.upper_max.max())

liquidity_by_ticks = context.run_model('uniswap-v3.get-liquidity-by-ticks', 
                                      {"address": "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
                                      "min_tick": pool_min_tick,
                                      "max_tick": pool_max_tick})

liquidity_by_ticks


In [ ]:
pool_min_tick = min(df_burn_min_max.lower_min.min(), df_mint_min_max.lower_min.min())
pool_max_tick = max(df_burn_min_max.upper_max.max(), df_mint_min_max.upper_max.max())

pool = Contract(address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')
 
tick_mapping = get_liquidity_in_ticks(pool, pool_min_tick, pool_max_tick)

df_pool2 = get_amount_in_ticks(pool, token0, token1, tick_mapping)

assert df_pool2.liquidity[df_pool2.liquidity < 0].shape[0] == 0

display((
    ('token0 balance: ', token0_bal, df_pool2.token0_locked.sum(), token0_bal - df_pool2.token0_locked.sum()),
    ('token1 balance: ', df_pool2.token1_locked.sum(), token1_bal, df_pool2.token1_locked.sum() - token1_bal),
))

df_pool2.plot(x = 'tick', y = ['liquidity', 'token0_locked', 'token1_locked'], sharex=True, subplots=True, kind='line')
plt.show()

In [ ]:
df_pool2.plot(x = 'tick', y = ['liquidity', 'token0_locked_scaled', 'token1_locked_scaled'], sharex=True, subplots=True, kind='line')
plt.show()

In [ ]:
df_pool2.token1_locked_scaled.sum(), token1.scaled(token1_bal)

In [ ]:
df_pool2.plot(x = 'tick', y = ['liquidity'], figsize=(20,4))
plt.vlines(x=current_tick, ymin=0, ymax=df_pool2.liquidity.max(), linestyles='dashed', colors='red')
plt.show()

df_pool2.query('(tick > 170000) & (tick < 220000)').plot(x = 'tick', y = ['liquidity'], figsize=(20,4))
plt.vlines(x=current_tick, ymin=0, ymax=1.75e19, linestyles='dashed', colors='red')
plt.show()

In [ ]:
ax = plt.gca()    # Get current axis
ax2 = ax.twinx()  # make twin axis based on x

df_pool2.query('(tick > @current_tick-@tick_spacing*10) & (tick < 220000)').plot(x = 'tick', y = ['token0_locked_scaled'], figsize=(20,4), ax=ax)
df_pool2.query('(tick < @current_tick-@tick_spacing*10) & (tick > 170000)').plot(x = 'tick', y = ['token1_locked_scaled'], figsize=(20,4), ax=ax2, color='green')
ax.vlines(x=current_tick, ymin=0, ymax=df_pool2.token0_locked_scaled.max(), linestyles='dashed', colors='red')
plt.show()

In [ ]:
pa = 2022
pb = 1358

p_a = (1 / pa / scale_multiplier)
p_b = (1 / pb / scale_multiplier)

tick_a = math.log(p_a) / math.log(1.0001)
tick_b = math.log(p_b) / math.log(1.0001)

ax = plt.gca()    # Get current axis
ax2 = ax.twinx()  # make twin axis based on x

df_pool2_sel = df_pool2.query('(tick > @tick_a) & (tick < @tick_b)')
df_pool2_sel

df_pool2_sel.plot(x = 'tick', y = ['token0_locked_scaled'], figsize=(20,4), ax=ax)
df_pool2_sel.plot(x = 'tick', y = ['token1_locked_scaled'], figsize=(20,4), ax=ax2, color='green')
ax.vlines(x=current_tick, ymin=0, ymax=df_pool2_sel.token0_locked_scaled.max(), linestyles='dashed', colors='red')
plt.show()

On block 15269596

![UniswapV3 USDC-WETH-POOL](pool_liquidity_usdc_weth.png)

In [ ]:
df_pool2.query('(tick > @tick_a) & (tick < @tick_b)')